# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [152]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [153]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [154]:
import os
from glob import glob
parquet_files = glob(os.path.join(os.getenv('PRICE_DATA'), "*/*/*.*"))

parquet_files

['../../05_src/data/prices/BKTI/BKTI_2012/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2012/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2014/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2014/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2013/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2013/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1980/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1980/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1987/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1987/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1989/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1989/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1988/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1988/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_1986/part.0.parquet

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [160]:
import pandas as pd
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px

dd_feat = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1))
                       .assign(Adj_Close_lag_1 = x['Adj Close'].shift(1))
                       .assign(hi_lo_range = x['High'] - x['Low']), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1': 'f8',
                    'hi_lo_range': 'f8'
                    },
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

dd_feat = dd_feat.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [161]:
dd_feat_PanDF = dd_feat.compute().assign(
    Mov_Avg_returns = lambda x: x['returns'].rolling(10).mean()
)
dd_feat_PanDF

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,hi_lo_range,returns,Mov_Avg_returns
ticker,,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,7.153078,NaN,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,2.280043,-0.082386,NaN
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,2.816525,0.089783,NaN
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,2.592991,-0.090909,NaN
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,1.385908,0.026563,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2020-03-26,9.610000,9.940000,9.260000,9.590000,9.590000,60500.0,ZEUS.csv,2020,9.670000,9.670000,0.679999,-0.008273,0.001594
ZEUS,2020-03-27,9.330000,9.330000,8.700000,8.700000,8.700000,52900.0,ZEUS.csv,2020,9.590000,9.590000,0.630000,-0.092805,-0.006110
ZEUS,2020-03-30,8.810000,9.760000,8.700000,9.680000,9.680000,73700.0,ZEUS.csv,2020,8.700000,8.700000,1.060000,0.112644,0.003339


In [162]:
#Alternative method
dd_feat_PanDF = dd_feat.compute()
dd_feat_PanDF['Mov_Avg_returns'] = dd_feat_PanDF['returns'].rolling(10).mean()
dd_feat_PanDF

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,hi_lo_range,returns,Mov_Avg_returns
ticker,,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,7.153078,NaN,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,2.280043,-0.082386,NaN
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,2.816525,0.089783,NaN
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,2.592991,-0.090909,NaN
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,1.385908,0.026563,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2020-03-26,9.610000,9.940000,9.260000,9.590000,9.590000,60500.0,ZEUS.csv,2020,9.670000,9.670000,0.679999,-0.008273,0.001594
ZEUS,2020-03-27,9.330000,9.330000,8.700000,8.700000,8.700000,52900.0,ZEUS.csv,2020,9.590000,9.590000,0.630000,-0.092805,-0.006110
ZEUS,2020-03-30,8.810000,9.760000,8.700000,9.680000,9.680000,73700.0,ZEUS.csv,2020,8.700000,8.700000,1.060000,0.112644,0.003339


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Yes, it was necessary to convert to pandas as doing in dask requires repartioning due to our default partiton size being less.
Pandas fits the entire data set into RAM for operations so it was easy to perform rolling average calculation.

+ Would it have been better to do it in Dask? Why?

Yes with Large datasets (100 GB to 100 TB) that may exceed a single machine's memory. It is best to use Dask as it Parallelizes operations across multiple CPU cores on one machine or a cluster of machines. Thus, it is faster for large datasets and long computations, where parallelism provides significant speedups.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.